In [54]:
#!/usr/bin/env python3

import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.externals import joblib
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

import warnings
warnings.filterwarnings('ignore')

In [75]:
class Data():
    def __init__(self, train_csv, train_target_csv, cat_features, num_features, target):
        self.cat_features = cat_features
        self.num_features = num_features
        self.target_label = target
        self.data_frame = self.read_data(train_csv, output="features")
        self.target_data = self.read_data(train_target_csv, output="target")
        self.feature_names = list(self.data_frame.columns.values)
        self.Xtrain = None
        self.ytrain = None
        self.Xtest = None
        self.ytest = None
        self.pre_process()
        self.split_data()
        
    def standardize_column_names(self):
        old_column_names = self.feature_names
        new_column_names = [column_name.lower() for column_name in old_column_names]
        self.feature_names = new_column_names
        self.data_frame.columns = self.feature_names

    def convert_to_cat_dtype(self):
        for feat in self.cat_features:
            self.data_frame[feat] = self.data_frame[feat].astype("category")
    
    def scale_numeric_features(self):
        num_data = self.data_frame[self.num_features].values
        std_scaler = StandardScaler()
        scaled_values = std_scaler.fit_transform(num_data)
        num_df = pd.DataFrame(scaled_values, columns=self.num_features)
        self.data_frame = pd.concat([num_df, self.data_frame[self.cat_features]], axis=1)
        
    def encode_cat_features(self):
        new_df = self.data_frame[self.num_features].copy()
        for cat_feature in self.cat_features:
            encoded_output = pd.get_dummies(self.data_frame[cat_feature])
            new_df = pd.concat([new_df, encoded_output], axis=1)
        self.data_frame = new_df
            
            
    def pre_process(self):
        self.standardize_column_names()
        self.convert_to_cat_dtype()
        self.scale_numeric_features()
        self.encode_cat_features()
        
    def read_data(self, csv, output="features"):
        df = pd.read_csv(csv)
        if output=="features":
            return df.iloc[:, 1:]
        else:
            target = df.loc[:,self.target_label]
            return target
    
    def split_data(self):
        consolidated_df = pd.concat([self.data_frame, self.target_data],axis=1)
        train, test = train_test_split(consolidated_df, test_size=0.2)
        self.Xtrain = train.iloc[:, :-1]
        self.ytrain = train.iloc[:,-1]
        self.Xtest = test.iloc[:, :-1]
        self.ytest = test.iloc[:,-1]


class Models:
    def __init__(self, data):
        self.Xtrain = data.Xtrain
        self.ytrain = data.ytrain
        self.Xtest = data.Xtest
        self.ytest = data.ytest
        self.models = []
        self.model_info = {}
        self.predictions = None
        self.best_model = None
        
    
    def add_model(self, model):
        self.models.append(model)
    
    def cross_validate(self, k):
        for model in self.models:
            cross_evaluation = cross_val_score(model, self.Xtrain, self.ytrain, cv=k, scoring="neg_mean_squared_error")
            score = np.mean(np.negative(cross_evaluation))
            self.model_info[model] = score
        
    def select_best_model(self):
        self.best_model = min(self.modelInfo, key=self.modelInfo.get)
        
    def summarize(self):
        for model in self.models:
            print(model)
            print(print(self.model[model]))
            print(self.best_model)
            
    def run(self):
        self.cross_validate(5)
        self.select_best_model()
        self.summarize()
        
            
    
        
            

In [ ]:
dataset_features = "data/train_features.csv"
dataset_target = "data/train_salaries.csv"
cat_features = ["companyid", "jobtype", "degree", "major", "industry"]
num_features = ["milesfrommetropolis", "yearsexperience"]

lin_alg = LinearRegression()
ridge_reg = Ridge()
rfr = RandomForestRegressor()
gbr = GradientBoostingRegressor()

if __name__ == '__main__':
    data = Data(dataset_features, dataset_target,cat_features, num_features, "salary")
    print(data.Xtrain.shape)
    print(data.Xtest.shape)
    print(data.ytrain.shape)
    print(data.ytest.shape)
    models = Models(data)
    models.add_model(lin_alg)
    models.add_model(ridge_reg)
    models.add_model(rfr)
    models.add_model(gbr)
    models.run()


(800000, 94)
(200000, 94)
(800000,)
(200000,)
